In [2]:
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)

[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import logging
from kafka import KafkaConsumer
import requests
import json
import psycopg2
import uuid

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

resp = []

# Replace with your Kafka broker address(es)
brokers = ["*:9092"]

# Replace with your topic name
topic = "three"
# KServe inference service URL
kserve_url = 'http://dvc.kubeflow-user-example-com.svc.cluster.local/v1/models/dvc:predict'

# Create a Kafka consumer
logger.info("Creating Kafka consumer...")
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=brokers,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-group',
    value_deserializer=lambda v: v.decode('utf-8')
)

logger.info(f"Subscribed to topic: {topic}")


# Function to send data to KServe
def send_to_kserve(data):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(kserve_url, headers=headers, data=json.dumps(data))
    return response.json()

def outcome_to_database(values):
    db_details = {
        'dbname': '*',
        'user': '*',
        'password': '*',
        'host': '*',
        'port': 5432
    }
    # Data to insert
    outcome_data = {
        'uid': values[0],  # Generating a unique ID
        'outcome': values[1]  # This can be 0 or 1
    }

# Connect to PostgreSQL
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        exit()

    # Insert data into the table
    try:
        insert_query = """
        INSERT INTO outcomes (uid, outcome)
        VALUES (%s, %s)
        """
        cursor.execute(insert_query, (outcome_data['uid'], outcome_data['outcome']))
        conn.commit()
        print("Data inserted successfully.")
    except Exception as e:
        print(f"Failed to insert data: {e}")
        conn.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("PostgreSQL connection closed.")

        
        

try:
    for message in consumer:
        try:
            resp = []
            message = json.loads(message.value)  # Message value is already a dict
            print(f"Received message: {message}")
            uid = message.pop('uid', None)
            resp.append(uid)
            #print(f"new message: {message}")
            # Prepare data for KServe
            kserve_payload = {
                "instances": [list(message.values())]  # Reshape the data to 2D array
            }

            # Send data to KServe
            kserve_response = send_to_kserve(kserve_payload)
            resp.append(kserve_response['predictions'][0])
            print(f"KServe response: {kserve_response}")
            print(resp)
            outcome_to_database(resp)
        except json.JSONDecodeError:
            print(f"Received non-JSON message: {message.value}")
        except Exception as e:
            logger.error(f"Error sending to KServe: {e}")
        #logger.info(f"Partition: {message.partition}, Offset: {message.offset}")
except KeyboardInterrupt:
    logger.info("Consumer stopped manually.")
finally:
    consumer.close()
    logger.info("Consumer closed.")


INFO:__main__:Creating Kafka consumer...
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=ec2-3-84-88-216.compute-1.amazonaws.com:9092 <connecting> [IPv4 ('3.84.88.216', 9092)]>: connecting to ec2-3-84-88-216.compute-1.amazonaws.com:9092 [('3.84.88.216', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=ec2-3-84-88-216.compute-1.amazonaws.com:9092 <connecting> [IPv4 ('3.84.88.216', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('three',)
INFO:__main__:Subscribed to topic: three
INFO:kafka.cluster:Group coordinator for my-group is BrokerMetadata(nodeId='coordinator-0', host='ec2-3-84-88-216.compute-1.amazonaws.com', port=9092, rack=None)
INFO:kafka.coordinator:Discovered coordinator coordinator-0 

Received message: {'SizeOfOptionalHeader': -0.36050749977742014, 'MinorLinkerVersion': -0.37628545401339736, 'SizeOfCode': 0.7319818837332536, 'SizeOfInitializedData': -0.21890447616440128, 'SizeOfUninitializedData': -0.12754141233745858, 'BaseOfCode': -0.10815786799743762, 'BaseOfData': 0.8442979845835227, 'ImageBase': -0.014265954069590235, 'SectionAlignment': -0.02785883454533931, 'MajorOperatingSystemVersion': -3.783301043371495, 'MinorImageVersion': -0.08601248526438687, 'MajorSubsystemVersion': -8.146929400017378, 'SizeOfHeaders': -0.19993743506016834, 'CheckSum': -0.30014776191072096, 'Subsystem': 18.955819550476978, 'DllCharacteristics': -1.18386423766959, 'SizeOfStackReserve': 0.36320900044581794, 'SizeOfStackCommit': -0.18911824926091036, 'SizeOfHeapReserve': 0.1275489610136435, 'NumberOfRvaAndSizes': -0.006541198444001456, 'SectionsNb': 1.685708547402764, 'SectionsMinEntropy': 0.7753814443163086, 'SectionsMaxEntropy': 0.2730052092310991, 'SectionMaxRawsize': 0.11133533433969

INFO:__main__:Consumer stopped manually.
INFO:kafka.coordinator:Stopping heartbeat thread
INFO:kafka.coordinator:Leaving consumer group (my-group).
INFO:kafka.conn:<BrokerConnection node_id=coordinator-0 host=ec2-3-84-88-216.compute-1.amazonaws.com:9092 <connected> [IPv4 ('3.84.88.216', 9092)]>: Closing connection. 
INFO:kafka.conn:<BrokerConnection node_id=2 host=ec2-3-84-88-216.compute-1.amazonaws.com:9094 <connected> [IPv4 ('3.84.88.216', 9094)]>: Closing connection. 
INFO:kafka.conn:<BrokerConnection node_id=0 host=ec2-3-84-88-216.compute-1.amazonaws.com:9092 <connected> [IPv4 ('3.84.88.216', 9092)]>: Closing connection. 
ERROR:kafka.consumer.fetcher:Fetch to node 0 failed: Cancelled: <BrokerConnection node_id=0 host=ec2-3-84-88-216.compute-1.amazonaws.com:9092 <connected> [IPv4 ('3.84.88.216', 9092)]>
INFO:__main__:Consumer closed.
